# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll then also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

You will be able to:
* Use ANOVA for testing multiple pairwise comparisons
* Understand and explain the methodology behind ANOVA tests

## Loading the Data

Start by loading in the data stored in the file **ToothGrowth.csv**.

In [2]:
# Your code here

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generating the ANOVA Table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage 

In [3]:
#Your code here
formula = 'len ~ C(supp) + dose'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Reading the Table

Make a brief comment regarding the statistics regarding the effect of supplement and dosage on tooth length.

In [4]:
# Your comment here
# Both factors are influential, with dose seeming to be the most influential.

## Comparing to T-Tests

Now that you've gotten a brief chance to interact with ANOVA, its interesting to compare the results to those from the t-tests you were just working with. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterwards, you'll conduct a t-test to compare the tooth length of these two different samples.

In [19]:
#Your code here

oj_df = df[df.supp == 'OJ']['len']
vc_df = df[df.supp == 'VC']['len']


Now compare a t-test between these two groups and print the associated two-sided p-value.

In [16]:
#Your code here; calculate the 2-sided p-value for a t-test comparing the two supplement groups.
from scipy import stats
import numpy as np
""" Calculate Welch's t statistic for two samples. """
def welch_t(a, b):
    
    n1 = len(a)
    n2 = len(b)
    s1 = np.var(a, ddof=1)
    s2 = np.var(b, ddof=1)
    x1 = np.mean(a)
    x2 = np.mean(b)
    
    num = abs(x1-x2)
    denom = np.sqrt((s1/n1)+(s2/n2))
    
    return num/denom


""" Calculate the effective degrees of freedom for two samples. """
def welch_df(a, b):
    s1 = np.var(a, ddof=1)
    s2 = np.var(b, ddof=1)
    n1 = len(a)
    n2 = len(b)
    v1 = n1-1
    v2 = n2-1
    
    num = ((s1/n1)+(s2/n2))**2
    denom = (s1**2/((n1**2)*v1))+(s2**2/((n2**2)*v2))

    return num/denom

def p_value(a, b, two_sided=False):
    #Your code here
    t = welch_t(a,b)
    df = welch_df(a,b)
    p = 1-stats.t.cdf(t,df)
    
    if two_sided:
        p = p*2
    return p

In [20]:
p = p_value(vc_df, oj_df, two_sided=True)
p

0.06063450788093405

In [22]:
stats.ttest_ind(vc_df, oj_df)

Ttest_indResult(statistic=-1.91526826869527, pvalue=0.06039337122412849)

## A 2-Category ANOVA F-Test is Equivalent to a 2-Tailed t-Test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-Test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [23]:
#Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
#Compare the p-value to that of the t-test above. 
#They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Generating Multiple T-Tests

While the 2-category ANOVA test is identical to a 2-tailed t-Test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [6]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2 sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [24]:
#Your code here; reuse your $t$-test code above to calculate the p-value for a 2-sided $t$-test
#for all combinations of the supplement-dose groups listed above. 
#(Since there isn't a control group, compare each group to every other group.)

pairs ={}
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    pairs[group_name] = data
    

In [27]:
for i in pairs:
    for j in pairs:
        if i != j:
            p = p_value(pairs[i], pairs[j], two_sided=True)
            print(i, j, p)

('OJ', 0.5) ('OJ', 1.0) 8.784919055160323e-05
('OJ', 0.5) ('OJ', 2.0) 1.323783877626994e-06
('OJ', 0.5) ('VC', 0.5) 0.00635860676409683
('OJ', 0.5) ('VC', 1.0) 0.04601033257637588
('OJ', 0.5) ('VC', 2.0) 7.196253523966689e-06
('OJ', 1.0) ('OJ', 0.5) 8.784919055160323e-05
('OJ', 1.0) ('OJ', 2.0) 0.03919514204624397
('OJ', 1.0) ('VC', 0.5) 3.655206737285255e-08
('OJ', 1.0) ('VC', 1.0) 0.0010383758722998238
('OJ', 1.0) ('VC', 2.0) 0.09652612338267019
('OJ', 2.0) ('OJ', 0.5) 1.323783877626994e-06
('OJ', 2.0) ('OJ', 1.0) 0.03919514204624397
('OJ', 2.0) ('VC', 0.5) 1.3621326289126046e-11
('OJ', 2.0) ('VC', 1.0) 2.3610742028168374e-07
('OJ', 2.0) ('VC', 2.0) 0.9638515887233756
('VC', 0.5) ('OJ', 0.5) 0.00635860676409683
('VC', 0.5) ('OJ', 1.0) 3.655206737285255e-08
('VC', 0.5) ('OJ', 2.0) 1.3621326289126046e-11
('VC', 0.5) ('VC', 1.0) 6.811017703167721e-07
('VC', 0.5) ('VC', 2.0) 4.681577414622495e-08
('VC', 1.0) ('OJ', 0.5) 0.04601033257637588
('VC', 1.0) ('OJ', 1.0) 0.0010383758722998238
('

## Summary

In this lesson, you examined the ANOVA technique to generalize A/B testing methods to multiple groups and factors.